# Imports and Description

In [1]:
import numpy as np

First I will put together the model without any scenario analysis and with the normal economy inputs, then I will add the scenario analysis afterwards.

# Inputs

In [2]:
price = 12
output = 500000
int_rate = 0.05
project_cost = 10000000
variable_cost = 7
machine_life = 10

# Profit

First let's figure out how to get the profit in each year. It does not change over time so we can figure out a single year then expand it to the number of years.

Get the profit in a single year: $P = Q(P - C_v)$

In [3]:
profit = output * (price - variable_cost)
profit

2500000

Now get it in a list of machine_life number of years

In [4]:
all_profit = [profit] * machine_life
all_profit

[2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000]

Now wrap this up in a function.

In [5]:
def profits(output, price, variable_cost):
    profit = output * (price - variable_cost)
    all_profit = [profit] * machine_life
    return all_profit

profits(output, price, variable_cost)

[2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000]

# NPV

The first cash flow should be the machine cost, then the profits should come afterwards, and all that should be passed to NPV.

Put the initial outlay together with the cash flows.

In [6]:
all_profits = profits(output, price, variable_cost)
all_cash_flows = [-project_cost] + all_profits
all_cash_flows

[-10000000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000,
 2500000]

Now calculate the NPV

In [7]:
np.npv(int_rate, all_cash_flows)

9304337.322962027

Now let's wrap this section up in a function. This function actually also calculates profits by calling our previous function `profits`, and so now this function is the whole model.

In [8]:
def project_npv(output, price, variable_cost, project_cost, int_rate):
    all_profits = profits(output, price, variable_cost)
    all_cash_flows = [-project_cost] + all_profits
    npv = np.npv(int_rate, all_cash_flows)
    return npv

project_npv(output, price, variable_cost, project_cost, int_rate)

9304337.322962027

# Scenario Analysis

Now our model works for a single set of inputs. Let's use the external scenario analysis approach here. So we will run the model multiple times, once for each set of scenario inputs, then we will collect the outputs, and take the expected value of the outputs.

I will write a loop which calls the function for each set of inputs. But first I need to set up the inputs in a way that will be easy to call in a loop. Let's use a nested dictionary for this.

In [9]:
states_of_economy = {
    'Expansion': {
        'output': 1000000,
        'price': 15,
        'int_rate': 0.07,
        'probability': 0.2,
    },
    'Normal': {
        'output': 500000,
        'price': 12,
        'int_rate': 0.05,
        'probability': 0.7,
    },
    'Recession': {
        'output': 200000,
        'price': 10,
        'int_rate': 0.03,
        'probability': 0.1,
    },
}

I'll quickly show accesssing items and looping through this nested dictionary

In [10]:
states_of_economy['Expansion']['output']

1000000

In [11]:
for soe, input_dict in states_of_economy.items():
    print(soe)

Expansion
Normal
Recession


In [12]:
for soe, input_dict in states_of_economy.items():
    print(input_dict)

{'output': 1000000, 'price': 15, 'int_rate': 0.07, 'probability': 0.2}
{'output': 500000, 'price': 12, 'int_rate': 0.05, 'probability': 0.7}
{'output': 200000, 'price': 10, 'int_rate': 0.03, 'probability': 0.1}


In [13]:
for soe, input_dict in states_of_economy.items():
    print(input_dict['output'])

1000000
500000
200000


Now let's build the loop which runs the model multiple times. We will use another dictionary to store the results.

In [14]:
output_dict = {}
for soe, input_dict in states_of_economy.items():
    npv = project_npv(
        input_dict['output'],
        input_dict['price'],
        variable_cost,
        project_cost,
        input_dict['int_rate']
    )
    output_dict[soe] = npv
output_dict

{'Expansion': 46188652.327460796,
 'Normal': 9304337.322962027,
 'Recession': -4881878.297934502}

Therefore the business is a failure in the recession, but a success in the other states of the economy.

One more loop to calculate expected value. This could have been done in the prior loop as well, but I separated them to make it more clear what's going on. Here we are calculating $E[X] = \sum_{i=1}^N \frac{x_i}{N}$

In [15]:
ev = 0
for soe, input_dict in states_of_economy.items():
    npv = output_dict[soe]  # look up previously stored result for this economy case
    ev = ev + npv * input_dict['probability']
ev

15262578.761772128

Wrapping this all up in a function

In [16]:
def project_npv_for_input_cases(input_cases, variable_cost, project_cost):
    output_dict = {}
    for soe, input_dict in states_of_economy.items():
        npv = project_npv(
            input_dict['output'],
            input_dict['price'],
            variable_cost,
            project_cost,
            input_dict['int_rate']
        )
        output_dict[soe] = npv
    print(output_dict)
    
    ev = 0
    for soe, input_dict in states_of_economy.items():
        npv = output_dict[soe]  # look up previously stored result for this economy case
        ev = ev + npv * input_dict['probability']
    
    return ev

project_npv_for_input_cases(states_of_economy, variable_cost, project_cost)

{'Expansion': 46188652.327460796, 'Normal': 9304337.322962027, 'Recession': -4881878.297934502}


15262578.761772128

Same thing, but now adding some output formatting.

In [17]:
def project_npv_for_input_cases(input_cases, variable_cost, project_cost):
    output_dict = {}
    for soe, input_dict in states_of_economy.items():
        npv = project_npv(
            input_dict['output'],
            input_dict['price'],
            variable_cost,
            project_cost,
            input_dict['int_rate']
        )
        output_dict[soe] = npv
    
    ev = 0
    for soe, input_dict in states_of_economy.items():
        npv = output_dict[soe]  # look up previously stored result for this economy case
        ev = ev + npv * input_dict['probability']
        
    for soe, npv in output_dict.items():
        print(f'NPV for a {soe} is ${npv:,.0f}')
    print(f'Expected Value: ${ev:,.0f}')
    
    return ev

project_npv_for_input_cases(states_of_economy, variable_cost, project_cost);

NPV for a Expansion is $46,188,652
NPV for a Normal is $9,304,337
NPV for a Recession is $-4,881,878
Expected Value: $15,262,579
